<a href="https://colab.research.google.com/github/caesarcc/pucminas-tcc-fake-news-detection/blob/main/passo02_preparacao_do_PTT5_sumarizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação do Modelo T5, pré-treinado em PT-BR, para sumarização

### Iniciando pela configuração do Google Colab, libs externas e acesso ao Google Drive

In [11]:
# lib para utilizar arquitetura transformers
!pip install -q transformers==2.9.0
# garantir o uso da lib de pytorch testada neste projeto
!pip install -q pytorch_lightning==0.7.5
# lib necessária para o tokenizador   
!pip install -q sentencepiece
# lib auxiliar para facilitar o scraping de artigos na web
!pip install -q newspaper3k

     |████████████████████████████████| 635 kB 14.2 MB/s 
     |████████████████████████████████| 880 kB 60.7 MB/s 
     |████████████████████████████████| 5.6 MB 47.9 MB/s 
     |████████████████████████████████| 233 kB 18.3 MB/s 
     |████████████████████████████████| 829 kB 75.6 MB/s 


In [12]:
# Acesso ao Google Driver onde serão salvos os arquivos grandes
# Atenção: Todos arquivos usados também estão disponíveis nas pastas modelos e dados, respectivamente
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Importação de bibliotecas
import re
import newspaper
from transformers import T5Tokenizer
from transformers import T5Model, T5ForConditionalGeneration

### Carga dos modelos pré-treinados em arquitetura transformers, disponíveis na plataforma [Hugging Face](https://huggingface.co/models)

In [14]:
# Modelo trainado em PT-BR
model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'
# Tokenizador para ptbr
tokenizer = T5Tokenizer.from_pretrained(model_name)
# Pesos do modelo em PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(model_name)

TypeError: ignored

In [ ]:
artigo = newspaper.Article('https://www.e-farsas.com/a-foto-da-capa-da-revista-time-com-o-ex-presidente-lula-e-real.html', language='pt')
artigo.download()
artigo.parse()
print(artigo.text[:200])

- Publicidade -

Será que a foto mostrando o ex-presidente Lula na capa de maio de 2022 da revista Time é verdadeira ou falsa?

A imagem começou a se espalhar nas redes sociais, além de ser bastante c


In [ ]:
# Limpar textos
def limpar_textos(texto):
    texto = str(texto).replace(r'http[\w:/\.]+', ' ')  # remover urls
    texto = str(texto).replace(r'[^\.\w\s]', ' ')  # remover tudo exceto caracters e pontuação
    texto = str(texto).replace('[^a-zA-Z]', ' ')
    texto = str(texto).replace(r'\s\s+', ' ')
    # Remoção de texto entre caracteres de destaque [], () ou --
    texto = re.sub("[-\(\[].*?[\)\]-]", "", str(texto))
    # Corrige novas linhas indevidas
    texto = str(texto).replace(r'\n', ' ').replace('\n', ' ')
    # Correção das pontuações duplicadas, há muitos casos no dadaset
    texto = re.sub(r'(\W)(?=\1)', '', str(texto)).replace(". .", ".")
    return texto.strip()

artigo_limpo = limpar_textos(artigo.text)
artigo_limpo[:200]

'Será que a foto mostrando o ex-presidente Lula na capa de maio de 2022 da revista Time é verdadeira ou falsa? A imagem começou a se espalhar nas redes sociais, além de ser bastante compartilhada em gr'

In [ ]:
tokens_entrada = tokenizer.encode(artigo_limpo, return_tensors="pt", max_length=400, truncation=True)

# max_length: O número máximo de tokens a serem gerado.
# length_penalty: Penalidade exponencial para o comprimento, 1,0 significa sem penalidade.
# num_beams: Especifica como usar busca heurística (beam search) em vez de busca gulosa (greedy search), 
#            O modelo tentará manter as 4 hipóteses mais prováveis ​​em cada passo de tempo.
# early_stopping: A geração seja concluída quando todas as hipóteses (beams) atingirem o final do token de string ( EOS ).

outputs = model_pt.generate(
    tokens_entrada, 
    max_length=400, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
print(outputs)

tensor([[    0, 19066,    13,     7,  3860,  6011,     9,   239,    14,  2670,
         10276,    22,  2666,     4,   321,     4,   318,  2570,    11,   752,
          4714,    21,  3787,    52,  7931,  1854,    25,  1526,   348,     7,
            35,  6322,    33,   117,  2964,  1457,     3,   275,     4,    57,
          1125, 18455,    12,   769,    10, 14341,     6,    89,  3006,     8,
            10,  3659, 15455,     3,    22,   100,  1082,     4,   321,     4,
           318,  2570,     3,     8,  1624,     9,   239,  2013,  4714,     5,
           544,   345,    11,  3224,   674,   146,    12,   792,   304,   175,
           176,  4780,     4, 10276,    46,     9,  1366,    39,   860,    10,
           112,  1445, 10276,  1565, 16930,    16,   150,   157,  8816,     8,
          4565,  2618,  5064,     5,   461,   643,    47,    23,     7,   100,
           155,    13,  5585,  7931,     6,     4,  2666,     6,    11,  4714,
            18,   409,     6,  2633,  8371,    98,  

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))

Será que a foto mostrando o ex-presidente Lula na capa de maio de 2022 da revista Time é verdadeira ou falsa? A imagem começou a se espalhar nas redes sociais, além de ser bastante compartilhada em grupos do WhatsApp e do Telegram, na primeira semana de maio de 2022, e mostra o examericana Time. Ao lado da manchete em inglês “O segundo ato de Lula: o líder mais popular do Brasil”, Lula aparece vestindo um terno escuro e camisa azul clara. Como essa não foi a primeira vez que fotos falsas de capas da Time com presidentes brasileiros circularam pela web, muita gente ficou em dúvida se dessa vez era verdade ou não. Será que essa capa é real? Verdade ou mentira? A capa é real e sua divulgação foi feita pela Time em seu perfil no Instagram no dia 04 de maio de 2022: Ver essa foto no Instagram Uma publicação compartilhada por TIME Por que Lula está na capa da Time? Segundo a Time, Lula foi entrevistado no final de março de 2022 em São Paulo e o ex-presidente falou sobre diversos assuntos, co

### Testando

In [ ]:
CAMINHO_MODELO = "drive/MyDrive/PUC/TCC/modelos/t5_sumarizar"
model_pt.save_pretrained(f"{CAMINHO_MODELO}")
tokenizer.save_pretrained(f"{CAMINHO_MODELO}")

('drive/MyDrive/PUC/TCC/modelos/t5_sumarizar/tokenizer_config.json',
 'drive/MyDrive/PUC/TCC/modelos/t5_sumarizar/special_tokens_map.json',
 'drive/MyDrive/PUC/TCC/modelos/t5_sumarizar/spiece.model',
 'drive/MyDrive/PUC/TCC/modelos/t5_sumarizar/added_tokens.json')